<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 优化视频 AI 应用 #
视频 AI 应用的效力很大程度上取决于视频 AI 模型的推理性能。到目前为止，我们已经能够使用 TAO 工具包训练视频 AI 模型，但我们尚未考虑推理性能。确保 DeepStream 工作流平稳运行且不出现延迟是我们需要考虑的重要因素。此外，推理性能应可支持将视频 AI 应用部署到计算能力较低的边缘设备上。完整的模型训练工作流程包括在训练模型后进行优化，以便在部署前可以利用剪枝和量化等性能强大的功能。

<img src='images/optimized_pre-trained_model_workflow.png' width=1080>

## 学习目标 ##
在此 Notebook 中，您将学习如何使用 TAO 工具包来优化模型的推理性能，学习内容包括：
* 构建多源 DeepStream 工作流
* 微调视频 AI 模型以部署到 DeepStream
* 剪枝经训练的 Detectnet_v2 模型
* 使用量化感知训练

**目录** 
<br>
本 Notebook 涵盖以下部分：
1. [多源 DeepStream 工作流](#s1)
    * [练习 #1 - 构建使用多源的 DeepStream 工作流](#e1)
2. [优化视频 AI 模型进行推理](#s2)
    * [模型剪枝](#s2.1)
    * [评估已剪枝的模型](#s2.2)
    * [练习 #2 - 模型比较](#e2)
    * [使用量化感知训练重新训练经剪枝的模型](#s2.3)
    * [练习 #3 - 将经剪枝的模型转换为 QAT 并重新训练](#e3)
3. [评估经过重新训练的模型](#s3)
4. [导出校正缓存](#s4)
5. [部署到 DeepStream](#s5)

执行以下单元，为 TAO 工具包设置目录。

In [ ]:
# DO NOT CHANGE THIS CELL
# Set and create directories for the TAO Toolkit experiment
import os

!mkdir logs
os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

<a name='s1'></a>
## 多源 DeepStream 工作流 ##
DeepStream SDK 支持构建具有多个输入视频流的工作流。当存在多个输入源时，必须为每个源配备自己的解码器并将其链接到 `Gst-nvstreammux`。`Gst-nvstreammux` 插件又名为**多路复用器**，可将多个输入源的内容组合成批量帧。在将源连接到多路复用器时，必须使用带有衬垫 (Pad) 模板 `sink_%u` 的 `get_request_pad()`从该多路复用器请求新的衬垫 (Pad)。多路复用器会将 `<batch-size>` 帧组合成批处理缓冲区，而此缓冲区需使用 `set_property()` 加以指定。如果多路复用器的输出格式和输入格式相同，则多路复用器将来自该源的帧作为其输出批处理缓冲区的一部分转发。如果分辨率不同，则多路复用器会对帧执行相应比例的缩放，将其从输入移至批处理缓冲区。在系统将批次中的所有帧推送到下游时，多路复用器会使这些帧保持相同的分辨率。

<a name='e1'></a>
#### 练习 #1 - 构建使用多源的 DeepStream 工作流 ####
为演示包含多个输入的 DeepStream 工作流，我们创建了一个具有以下体系架构的示例应用 [app_04.py](sample_apps/app_04.py)。此工作流与我们目前已构建的工作流非常相似，我们仅在其中作出了一些修改：
1. 获取 _1_ 个视频文件，并将其用于任意数量的文件源 (`filesrc`)。
2. 使用平铺器 (`Gst-nvmultistreamtiler`) 从批处理缓冲区合成一个 2D 图块，为该图块设置 `rows`、`columns`、`width` 和 `height` 属性。
3. 使用我们在上一个 Notebook 中构建的物体检测模型。
4. 已将探针回调函数附加到平铺器的源衬垫 (Pad)。

我们可以通过执行脚本并传递 4 个参数来运行工作流，具体如下所示：<br> `python sample_apps/app_04.py <path to input h264 video> <path to nvinfer config file> <number of file sources> <name of output file>`。

<p><img src='images/multi_input_pipeline.png' width=1080></p>

**说明**：<br>
* 查看 [app_04.py](sample_apps/app_04.py) 的代码。
* 仅修改 `<FIXME>` 以创建要与 `Gst-nvstreammux` 相连接的必要元素，并根据所传递的参数执行迭代。请**保存对文件的更改**。
* 执行以下单元以检查 nvinfer config 文件、运行 DeepStream 工作流并查看 `nvdia-smi` 日志。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the nvinfer config file
!cat $SPEC_FILES_DIR/pgie_config_trafficcamnet_retrained.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# Run the app_04.py DeepStream pipeline w/ the custom ResNet18 model
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 20 > '/dli/task/logs/smi.log' & \
python sample_apps/app_04.py /dli/task/data/sample_30.h264 \
                            /dli/task/spec_files/pgie_config_resnet18_detector_unpruned.txt \
                            8 \
                            output_tiled.mp4

In [ ]:
# DO NOT CHANGE THIS CELL
# Convert the output video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_tiled.mp4 output_tiled_conv.mp4 \
        -y \
        -loglevel quiet

In [ ]:
# DO NOT CHANGE THIS CELL
# Show video
from IPython.display import Video
Video('output_tiled_conv.mp4', width=720)

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the smi.log
!cat logs/smi.log

In [ ]:
# for i in range(number_sources): 
#     print('Creating source_bin ', i, end='\r')
#     source=Gst.ElementFactory.make('filesrc', 'file-source_%u'%i)
#     source.set_property('location', args[1])
#     h264parser=Gst.ElementFactory.make('h264parse', 'h264-parser_%u'%i)
#     decoder = Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder_%u"%i)
#     pipeline.add(source)
#     pipeline.add(h264parser)
#     pipeline.add(decoder)
#     padname="sink_%u"%i
#     source.link(h264parser)
#     h264parser.link(decoder)
#     decodersrcpad=decoder.get_static_pad("src").link(streammux.get_request_pad(padname))

点击 ... 以显示**解决方案**。

**观察**：<br>
当我们使用当前未剪枝的模型处理多个输入流时，DeepStream 工作流的性能开始受到影响。
1. 在工作流运行的底部输出部分，可以看出运行 24 秒片段需要花费一段时间，这比单个输入的时间要长得多。工作流每秒处理的帧数不到 30 帧，而这即是输入流的处理速度。如果将此片段上线，将导致严重的延迟。请参阅 [GStreamer 关于阻塞探针的设计文档](https://gstreamer.freedesktop.org/documentation/additional/design/probes.html?gi-language=c#blocking-probes)，详细了解延迟发生的原因。
2. 我们还在日志 `nvidia-smi` 中发现，流多处理器在工作流运行期间的利用率非常高。

<a name='s2'></a>
## 优化视频模型以开展推理 ##
TAO 工具包提供多种功能来优化模型的推理性能，包括**剪枝**和**量化**。

<a name='s2.1'></a>
### 模型剪枝 ###
剪枝是通过微调模型以提升推理性能的方式之一，也是 TAO 工具包的独特的优势之一。从算法上讲，它把对整体准确性没有显著贡献的神经元从神经网络中删除掉。剪枝可显著减少模型的整体大小，从而降低内存占用量并提高推理吞吐量，这对边缘部署非常重要。模型剪枝步骤会在不经意中降低了模型的准确性。所以剪枝之后，我们需要紧接着使用相同的数据集重新训练模型以恢复丢失的准确率。

<p><img src='images/pruning.svg' width=540></p>

有关剪枝的更多信息，请参阅此 [NVIDIA 开发者博客文章](https://developer.nvidia.com/blog/transfer-learning-toolkit-pruning-intelligent-video-analytics/)。

使用 `prune` 子任务时，`-m` 参数表示预训练模型的路径，`-o` 参数表示输出文件的路径，`-k` 参数表示*加载（load）* 模型的密钥。一些可选参数包括：
* `-eq, --equalization_criterion`：将输入状态等同于元素级运算层或深度卷积层所采用的标准 _（arithmetic_mean、geometric_mean、union 和 intersection）_ ，默认值为 union。此参数适用于 _ResNets_ 和 _MobileNets_ 。
* `-pg, --pruning_granularity`：一次要删除的过滤器的数量 _（默认=8）_ 。
* `-pth`：用于比较归一化范数的阈值 _（默认=0.1）_ 。
* `-nf, --min_num_filters`：每个层需至少保留的过滤器的数量 _（默认=16）_ 。
* `-el, --excluded_layers`：被排除的层的列表 _（默认=[]）_ 。

通常，我们只需要调整 `-pth`（阈值），即可在准确性与模型大小之间进行取舍。`pth` 越高，模型越小（因此推理速度越高），但准确性越差。所用阈值视数据集而定。_0.1_ 的 `pth` 值只是起始值。如果重新训练的准确性良好，我们可以增加此值以获取更小的模型。否则，我们可以降低此值以获得更高的准确性。

In [ ]:
# DO NOT CHANGE THIS CELL
# View prune usage
!detectnet_v2 prune --help

In [ ]:
# DO NOT CHANGE THIS CELL
# Create a new ResNet model folder and prune the resnet18_detector model
!rm -rf $MODELS_DIR/resnet18_detector_pruned
!mkdir -p $MODELS_DIR/resnet18_detector_pruned

!detectnet_v2 prune -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.tlt \
                    -o $MODELS_DIR/resnet18_detector_pruned/resnet18_detector_pruned.tlt \
                    -k tlt_encode

In [ ]:
# DO NOT CHANGE THIS CELL
# List the model and sizes
!ls -rlt $MODELS_DIR/resnet18_detector/weights

!ls -rlt $MODELS_DIR/resnet18_detector_pruned

<a name='s2.22'></a>
### 评估已剪枝的模型 ###
模型剪枝后，准确性可能会降低，因为之前一些有帮助的权重可能已被移除。

执行以下单元，比较未剪枝模型与已剪枝模型的评估结果。

In [ ]:
# DO NOT CHANGE THIS CELL
# Evaluate the unpruned model
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt \
                       -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.tlt \
                       -k tlt_encode

In [ ]:
# DO NOT CHANGE THIS CELL
# Evaluate the pruned model
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt \
                       -m $MODELS_DIR/resnet18_detector_pruned/resnet18_detector_pruned.tlt \
                       -k tlt_encode

<a name='e2'></a>
#### 练习 #2 - 模型比较 ####
**说明**：
* 研究有关未剪枝和已剪枝模型的大小和平均精度均值 (mAP) 的输出。
* 请注意这两个模型的比较方式。

In [ ]:
##### WRITE ANSWERS HERE #####
#
# 
#
#
##############################

In [ ]:
##### WRITE ANSWERS HERE #####
#
# The pruned model is significantly smaller in size but has a lower mean average precision. 
#
#
##############################

点击 ... 以显示**解决方案**。

<a name='s2.3'></a>
### 使用量化感知训练来重新训练经过剪枝的模型 ###
为重新获得准确性，我们建议使用附带更新规格文件的 `train` 子任务（指向作为预训练模型文件的新剪枝模型），在同一数据集上重新训练此经过剪枝的模型。重新训练时需要考虑以下几点：
* 在重新训练剪枝模型时，应为 DetectNet_v2 关闭 `training_config` 中的 `regularizer` 选项以恢复准确性。通过将正则化类型设置为 `NO_REG`，即可完成此操作。其它所有参数均可保留在上一次训练的规格文件中。
* 应将 `model_config` 中的 `load_graph` 选项设为 `true`，以加载已剪枝的模型的图。
* 如果在重新训练后，模型显示 mAP 出现某种程度的下降，则说明对最初训练的模型而言，修剪的内容可能过多。请尝试降低剪枝阈值（以降低剪枝比率），并使用新模型重新训练。
* 作为一个 _可选项_ ，DetectNet_v2 支持**量化感知训练**，有助于优化模型。

深度神经网络 (DNN) 模型，例如常用的视频 AI 应用，通常在数据中心或私有/公共云中具有高端 GPU 的服务器上进行训练。此类系统通常使用**32位浮点数**执行运算，以利用权重的更大动态范围。但是，模型经过训练后，通常必须将其部署在计算资源和功耗预算较少的边缘硬件上。考虑到边缘的计算、内存和功耗限制，使用完整的 32 位表示法运行 DNN 推理对于实时分析来说并不实际。为帮助降低计算预算，同时不影响模型的结构和参数数量，我们可以用较低的参数精度运行推理。在许多情况下，不妨在权重中使用 **8 位整数**。而这样做所面临的挑战在于，训练后仅对权重进行舍入处理可能会导致模型的准确度较低，如果权重的动态范围较广则更是如此。虽然 8 位 **量化**非常适合节省计算和内存预算，但这是一个有损过程。在量化期间，小范围的浮点数将被压缩到固定数量的“信息桶”中，这会导致信息丢失。也就是说，最初使用 32个比特位所能表达的细微的差异现在已丢失，因为它们在被 8 个比特位表达时，就被量化在同一个桶中了。这就像是在将小数表示为整数时遇到的舍入错误。为在较低精度的推理过程中保持准确性，有必要尝试通过量化感知训练来减少因信息丢失而引起的错误。量化感知训练用于训练 DNN，以实现较低精度的 INT8 部署，同时不会影响准确性。这一训练会在训练模型时模拟推理时间量化，随后下游推理平台可以使用该模型生成的实际量化模型。在训练期间，我们会对量化权重和张量与 INT8 之间的误差执行建模，使模型能够适应并减轻误差。从技术层面而言，在量化感知训练期间，我们会对训练图中构建的模型作出如下修改：
1. 将现有节点替换为支持权重伪量化的节点。
2. 将现有激活函数转换为 ReLU-6（输出节点除外）。
3. 添加量化 Quantize 和去量化 De-Quantize (QDQ) 节点来计算中间张量的动态范围。

在训练期间，将经过计算的动态范围序列化为**缓存文件**，以在推理时使用。

<p><img src='images/qat_training.png' width=720></p>

请参阅[此处](https://developer.nvidia.com/blog/improving-int8-accuracy-using-quantization-aware-training-and-tao-toolkit/)，了解有关量化感知训练的更多信息。

<a name='e3'></a>
#### 练习 #3 - 将剪枝模型转换为 QAT 并重新训练 ####
通过将规格文件 `training_config` 组件中的 `enable_qat` 参数设置为 `true`，我们可将支持的模型转换为 QAT 模型。在创建用于重新训练的训练配置文件时，仅需更新 `training_config` 组件的 `enable_qat` 和 `regularizer`，以及 `model_config` 组件的 `pretrained_model_file` 和 `load_graph`。

**练习说明**：<br>
* 将 `<FIXME>` 更改为可接受的值，完成对训练配置文件 `model_config`[（单独修改此处的 qat 版本）](spec_files/model_config_qat.txt)部分和 `training_config`[（单独修改此处的 qat 版本）](spec_files/training_config_qat.txt)部分的修改。请**保存对文件的更改**。
* 执行以下单元，使用量化感知训练 (QAT) 重新训练经过剪枝的模型。

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/model_config_qat.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the config file
!cat $SPEC_FILES_DIR/training_config_qat.txt

In [ ]:
# model_config {
#   arch: "resnet"
#   pretrained_model_file: "/dli/task/tao_project/models/resnet18_detector_pruned/resnet18_detector_pruned.tlt"
#   load_graph: true
#   freeze_blocks: 0
#   freeze_blocks: 1
#   num_layers: 18
#   use_pooling: false
#   use_batch_norm: true
#   dropout_rate: 0.0
#   objective_set: {
#     cov: {}
#     bbox: {
#       scale: 35.0
#       offset: 0.5
#     }
#   }
# }

# training_config: {
#   batch_size_per_gpu: 16
#   num_epochs: 10
#   enable_qat: true
#   learning_rate: {
#     soft_start_annealing_schedule: {
#       min_learning_rate: 5e-6
#       max_learning_rate: 5e-4
#       soft_start: 0.1
#       annealing: 0.7
#     }
#   }
#   regularizer: {
#     type: L1
#     weight: 3e-9
#   }
#   optimizer: {
#     adam: {
#       epsilon: 1e-08
#       beta1: 0.9
#       beta2: 0.999
#     }
#   }
#   cost_scaling: {
#     enabled: false
#     initial_exponent: 20.0
#     increment: 0.005
#     decrement: 1.0
#   }
#   checkpoint_interval: 5
# }

点击 ... 以显示**解决方案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# UPDATED enable_qat and regularizer from training_config
# UPDATED pretrained_model_file and load_graph from model_config
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config_qat.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config_qat.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config_qat.txt
!cat $SPEC_FILES_DIR/combined_training_config_qat.txt

In [ ]:
# DO NOT CHANGE THIS CELL
# Initiate the training process
!detectnet_v2 train -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                    -r $MODELS_DIR/resnet18_detector_pruned_retrained_qat \
                    -k tlt_encode \
                    -n resnet18_detector_pruned_retrained_qat

In [ ]:
# DO NOT CHANGE THIS CELL
# List the newly retrained model
!ls -rlt $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights

<a name='s3'></a>
### 评估经过重新训练的模型 ###
重新训练完成后，我们可以评估经由量化感知训练 (QAT) 重新训练过的已剪枝模型。此模型的 mAP（平均精度均值）应与未剪枝模型（未执行 QAT）的精度不相上下。但是，由于量化，有时可能会看到 mAP 值下降。剪枝和重新训练可能是一个迭代过程，但 TAO 工具包可以轻松快速地制作不同版本的视频 AI 模型原型。

In [ ]:
# DO NOT CHANGE THIS CELL
# Evaluate the model using the same validation set as training
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                       -m $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights/resnet18_detector_pruned_retrained_qat.tlt \
                       -k tlt_encode

<a name='s4'></a>
## 导出带校正缓存的模型 ##
当我们对模型的准确性和推理性能充满信心时，便可以将其导出到 DeepStream 中。为了以较低精度启用推理并获得更好的性能，我们需要在 INT8 模式中生成 **TensorRT 引擎**。我们需要在此过程额外使用包含缩放因素的**缓存文件**，以帮助消除因低精度运算而出现的量化错误。我们也可以选择使用 `export` 子任务创建校正缓存。这即是我们所说的以 **INT8 模式**导出。在使用 `export` 子任务时，我们可以添加 `--cal_cache_file` 参数，以指明保存校正缓存文件的路径，并加入指示所需数据类型的 `--data_type int8` 参数。`--data_type` 参数可用的选项为 `fp32`、`fp16` 和 `int8`。如果不需要 INT8 模式的推理，默认值为 `fp32`。

执行以下单元以导出经过 QAT 训练的模型。此命令会通过经训练的模型生成 `.etlt` 文件，并将相应的 INT8 缩放因子序列化为 TensorRT 可读校正缓存文件。

In [ ]:
# DO NOT CHANGE THIS CELL
# Delete duplicate copies
!rm -rf $MODELS_DIR/resnet18_detector_final/resnet18_detector_pruned_retrained_qat.etlt
!rm -rf $MODELS_DIR/resnet18_detector_final/cal.bin

# Export the QAT trained model
!detectnet_v2 export -m $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights/resnet18_detector_pruned_retrained_qat.tlt \
                     -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                     -o $MODELS_DIR/resnet18_detector_final/resnet18_detector_pruned_retrained_qat.etlt \
                     -k tlt_encode \
                     --cal_cache_file $MODELS_DIR/resnet18_detector_final/cal.bin \
                     --data_type int8 \
                     --gen_ds_config

<a name='s5'></a>
### 部署到 DeepStream ###
经由 QAT 重新训练的已剪枝模型已准备就绪，可支持在 DeepStream 上加以部署。我们现在可以在 `Gst-nvinfer` 的配置文件中使用 INT8 模式的 `network-mode=1`。

执行以下单元以读取修改后的 `Gst-nvinfer config file`，并将其传递给 `app_04.py` 以运行 DeepStream 工作流。

In [ ]:
# DO NOT CHANGE THIS CELL
# Run the app_04.py DeepStream pipeline w/ the pruned ResNet18 model
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 20 > '/dli/task/logs/smi.log' & \
python sample_apps/app_04.py /dli/task/data/sample_30.h264 \
                            spec_files/pgie_config_resnet18_detector_optimized.txt \
                            16 \
                            output_tiled_optimized.mp4

In [ ]:
# DO NOT CHANGE THIS CELL
# Read the smi.log
!cat logs/smi.log

In [ ]:
# DO NOT CHANGE THIS CELL
# Convert the output video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_tiled_optimized.mp4 output_tiled_optimized_conv.mp4 \
        -y \
        -loglevel quiet

In [ ]:
# DO NOT CHANGE THIS CELL
# Show video
Video('output_tiled_optimized_conv.mp4', width=720)

**观察**：<br>
使用经剪枝的模型后，工作流运行平稳。该模型兼顾内存和硬件处理效率，能够准确执行来自多源的实时视频 AI 推理，且不会产生明显的延迟。

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>